In [1]:
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm
import random
import os

In [2]:
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

def add_noise(org_csv, noise_csv, save_path, snr_start, snr_end):
    org_files = pd.read_csv(org_csv)['path'].tolist()
    org_fs = pd.read_csv(org_csv)['id'].tolist()
    noise_files = pd.read_csv(noise_csv)['path'].tolist()

    for idx in tqdm(range(len(org_files))):
        audio, sr = librosa.load('data'+org_files[idx][1:])
        noise_file = random.choice(noise_files)
        noise, noise_sr = librosa.load('cut_noise/'+noise_file)

        if len(audio) < sr*5:
            padding = np.zeros((sr*5)- len(audio))
            audio = np.concatenate((audio, padding))

        audio = audio[:sr*5]
        noise = noise[:sr*5]
        # print(audio.shape)
        
        rms_signal = calculate_rms(audio)
        
        snr_db = random.uniform(snr_start, snr_end)
        snr_linear = 10**(snr_db / 20)
        rms_noise = rms_signal / snr_linear
        current_rms_noise = calculate_rms(noise)
        noise_scaling_factor = rms_noise / current_rms_noise
        adjusted_noise = noise * noise_scaling_factor
        noisy_audio = audio + adjusted_noise

        file_path = save_path + org_fs[idx] + '.ogg'
        
        sf.write(file_path, noisy_audio, sr, format='OGG')

In [3]:
org_path = 'padded_combined_train_for_noise.csv'
noise_path = 'noise_for_add.csv'
save_path = 'comb_noise_added/'

add_noise(org_path, noise_path, save_path, 0.7, 3)

100%|██████████| 30000/30000 [1:08:22<00:00,  7.31it/s]
